# Differential Item Functioning (DIF)

## 概要

**Differential Item Functioning (DIF: 特異項目機能)** とは、同じ能力を持つ異なる集団の受験者が、特定のテスト項目に対して異なる正答確率を示す現象である。

例えば、同じ数学的能力を持つ男女が、ある文章題に対して異なる正答率を示す場合、その項目にはDIFが存在する可能性がある。これは項目の内容が一方の集団に有利に働いていることを示唆し、テストの公平性（fairness）に関わる重要な問題である。

### DIFの種類

DIFには2種類ある：

**1. Uniform DIF（均一DIF）**

能力レベルに関わらず、一方の集団が常に有利（または不利）である場合。IRTの文脈では、項目困難度パラメータ $b$ が集団間で異なる状況に対応する。

**2. Non-uniform DIF（非均一DIF）**

集団間の差が能力レベルによって変化する場合。低能力では集団Aが有利だが、高能力では集団Bが有利になるなど。IRTでは識別力パラメータ $a$ が集団間で異なる状況に対応する。

## DIF検出手法

### Mantel-Haenszel法

最も広く使用されているDIF検出手法の一つ。能力の代理指標として総得点を用い、各得点層で2×2の分割表を作成し、集団間の正答オッズ比を検定する。

Mantel-Haenszel統計量：

$$
\chi^2_{MH} = \frac{\left[ \left| \sum_k (A_k - E(A_k)) \right| - 0.5 \right]^2}{\sum_k \text{Var}(A_k)}
$$

ここで $A_k$ は得点層 $k$ における参照群の正答者数、$E(A_k)$ はその期待値。

共通オッズ比の推定値：

$$
\hat{\alpha}_{MH} = \frac{\sum_k A_k D_k / N_k}{\sum_k B_k C_k / N_k}
$$

ETSでは、この対数を変換したDelta尺度（MH D-DIF）がよく使われる：

$$
\text{MH D-DIF} = -2.35 \ln(\hat{\alpha}_{MH})
$$

### ロジスティック回帰法

項目への正答を目的変数、能力（総得点）と集団を説明変数とするロジスティック回帰を用いる方法。

$$
\log \left( \frac{P(Y=1)}{1-P(Y=1)} \right) = \beta_0 + \beta_1 \theta + \beta_2 G + \beta_3 (\theta \times G)
$$

- $\theta$: 能力（または総得点）
- $G$: 集団を示すダミー変数
- $\beta_2 \neq 0$: Uniform DIFの存在
- $\beta_3 \neq 0$: Non-uniform DIFの存在

この方法の利点は、Uniform DIFとNon-uniform DIFを同時に検出できることである。

### IRT法

各集団で別々にIRTモデルを推定し、等化して尺度を揃えたのち、項目パラメータの差を検定する方法。

Lord's $\chi^2$ 検定：

$$
\chi^2 = (\hat{\boldsymbol{\xi}}_R - \hat{\boldsymbol{\xi}}_F)' \hat{\boldsymbol{\Sigma}}^{-1} (\hat{\boldsymbol{\xi}}_R - \hat{\boldsymbol{\xi}}_F)
$$

ここで $\hat{\boldsymbol{\xi}}$ は項目パラメータのベクトル、$\hat{\boldsymbol{\Sigma}}$ はその共分散行列の推定値。

### SIBTEST

Shealy & Stout (1993) が開発した、多次元性に基づくDIF検出法。項目をアンカー項目（DIFがないと仮定）と検討項目に分け、条件付き期待得点の差を検定する。

## Pythonでの実装

### difNLR パッケージ（R）

Rには `difR` や `difNLR` など、DIF分析に特化したパッケージがある。

```r
library(difNLR)

# Mantel-Haenszel法
difMH(Data, group, focal.name = 1)

# ロジスティック回帰法
difLogistic(Data, group, focal.name = 1, type = "both")
```

### Pythonによる実装例

PythonでのDIF分析は、`statsmodels` を用いたロジスティック回帰や、`scipy.stats` を用いたMantel-Haenszel検定で実装できる。

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import logit

In [ ]:
def mantel_haenszel_dif(response: np.ndarray, group: np.ndarray, total_score: np.ndarray):
    """
    Mantel-Haenszel法によるDIF検出
    
    Parameters
    ----------
    response : np.ndarray
        項目への応答（0/1）
    group : np.ndarray
        集団（0: 参照群, 1: 焦点群）
    total_score : np.ndarray
        総得点（能力の代理指標）
    
    Returns
    -------
    dict
        MH統計量、p値、共通オッズ比、MH D-DIF
    """
    # 得点層ごとに分割表を作成
    unique_scores = np.unique(total_score)
    
    A_sum = 0  # 参照群・正答
    B_sum = 0  # 参照群・誤答
    C_sum = 0  # 焦点群・正答
    D_sum = 0  # 焦点群・誤答
    
    numerator = 0
    denominator = 0
    expected_A = 0
    var_A = 0
    
    for score in unique_scores:
        mask = total_score == score
        
        A = np.sum((group[mask] == 0) & (response[mask] == 1))  # 参照群・正答
        B = np.sum((group[mask] == 0) & (response[mask] == 0))  # 参照群・誤答
        C = np.sum((group[mask] == 1) & (response[mask] == 1))  # 焦点群・正答
        D = np.sum((group[mask] == 1) & (response[mask] == 0))  # 焦点群・誤答
        
        N = A + B + C + D
        if N == 0:
            continue
        
        n1 = A + B  # 参照群の人数
        n2 = C + D  # 焦点群の人数
        m1 = A + C  # 正答者数
        m2 = B + D  # 誤答者数
        
        # オッズ比の計算用
        numerator += A * D / N
        denominator += B * C / N
        
        # MH統計量の計算用
        expected_A += n1 * m1 / N
        var_A += n1 * n2 * m1 * m2 / (N**2 * (N - 1)) if N > 1 else 0
    
    # 共通オッズ比
    alpha_mh = numerator / denominator if denominator > 0 else np.nan
    
    # MH D-DIF（ETS Delta尺度）
    mh_d_dif = -2.35 * np.log(alpha_mh) if alpha_mh > 0 else np.nan
    
    # MHカイ二乗統計量
    observed_A = np.sum((group == 0) & (response == 1))
    chi2_mh = (abs(observed_A - expected_A) - 0.5)**2 / var_A if var_A > 0 else np.nan
    p_value = 1 - stats.chi2.cdf(chi2_mh, df=1)
    
    return {
        'chi2_mh': chi2_mh,
        'p_value': p_value,
        'alpha_mh': alpha_mh,
        'mh_d_dif': mh_d_dif
    }

In [ ]:
def logistic_regression_dif(response: np.ndarray, group: np.ndarray, total_score: np.ndarray):
    """
    ロジスティック回帰法によるDIF検出
    
    Parameters
    ----------
    response : np.ndarray
        項目への応答（0/1）
    group : np.ndarray
        集団（0: 参照群, 1: 焦点群）
    total_score : np.ndarray
        総得点（能力の代理指標）
    
    Returns
    -------
    dict
        Uniform DIF検定結果、Non-uniform DIF検定結果
    """
    df = pd.DataFrame({
        'response': response,
        'group': group,
        'score': total_score,
        'interaction': group * total_score
    })
    
    # Model 1: 能力のみ
    model1 = logit('response ~ score', data=df).fit(disp=0)
    
    # Model 2: 能力 + 集団（Uniform DIFの検定）
    model2 = logit('response ~ score + group', data=df).fit(disp=0)
    
    # Model 3: 能力 + 集団 + 交互作用（Non-uniform DIFの検定）
    model3 = logit('response ~ score + group + score:group', data=df).fit(disp=0)
    
    # Uniform DIF: Model 1 vs Model 2
    lr_uniform = -2 * (model1.llf - model2.llf)
    p_uniform = 1 - stats.chi2.cdf(lr_uniform, df=1)
    
    # Non-uniform DIF: Model 2 vs Model 3
    lr_nonuniform = -2 * (model2.llf - model3.llf)
    p_nonuniform = 1 - stats.chi2.cdf(lr_nonuniform, df=1)
    
    # Total DIF: Model 1 vs Model 3
    lr_total = -2 * (model1.llf - model3.llf)
    p_total = 1 - stats.chi2.cdf(lr_total, df=2)
    
    return {
        'uniform_dif': {
            'chi2': lr_uniform,
            'p_value': p_uniform,
            'coefficient': model2.params.get('group', np.nan)
        },
        'nonuniform_dif': {
            'chi2': lr_nonuniform,
            'p_value': p_nonuniform,
            'coefficient': model3.params.get('score:group', np.nan)
        },
        'total_dif': {
            'chi2': lr_total,
            'p_value': p_total
        }
    }

### シミュレーションデータによる検証

In [ ]:
def simulate_dif_data(
    n_reference: int = 500,
    n_focal: int = 500,
    n_items: int = 20,
    dif_items: list[int] = None,
    dif_magnitude: float = 0.5,
    seed: int = 42
):
    """
    DIFを含むシミュレーションデータを生成
    
    Parameters
    ----------
    n_reference : int
        参照群の人数
    n_focal : int
        焦点群の人数
    n_items : int
        項目数
    dif_items : list[int]
        DIFを持つ項目のインデックス
    dif_magnitude : float
        DIFの大きさ（困難度パラメータの差）
    seed : int
        乱数シード
    
    Returns
    -------
    tuple
        応答データ、集団、総得点、真のDIF項目
    """
    np.random.seed(seed)
    
    if dif_items is None:
        dif_items = [0, 1]  # 最初の2項目にDIFを設定
    
    n_total = n_reference + n_focal
    
    # 能力パラメータ（両群とも標準正規分布）
    theta = np.concatenate([
        np.random.normal(0, 1, n_reference),
        np.random.normal(0, 1, n_focal)
    ])
    
    # 集団
    group = np.concatenate([np.zeros(n_reference), np.ones(n_focal)]).astype(int)
    
    # 項目パラメータ（1PLMを仮定）
    b = np.random.uniform(-2, 2, n_items)
    
    # 応答データの生成
    responses = np.zeros((n_total, n_items))
    
    for j in range(n_items):
        b_j = b[j]
        
        # DIF項目の場合、焦点群に対して困難度を変更
        if j in dif_items:
            b_effective = np.where(group == 1, b_j + dif_magnitude, b_j)
        else:
            b_effective = b_j
        
        # 正答確率（1PLM）
        prob = 1 / (1 + np.exp(-(theta - b_effective)))
        responses[:, j] = (np.random.random(n_total) < prob).astype(int)
    
    # 総得点
    total_score = responses.sum(axis=1)
    
    return responses, group, total_score, dif_items

In [ ]:
# シミュレーションデータの生成
responses, group, total_score, true_dif_items = simulate_dif_data(
    n_reference=500,
    n_focal=500,
    n_items=20,
    dif_items=[0, 1],
    dif_magnitude=0.8
)

print(f"データサイズ: {responses.shape}")
print(f"参照群: {np.sum(group == 0)}, 焦点群: {np.sum(group == 1)}")
print(f"真のDIF項目: {true_dif_items}")

In [ ]:
# 各項目についてDIF検出を実行
results = []

for j in range(responses.shape[1]):
    # 検討項目を除いた総得点を計算
    score_without_item = total_score - responses[:, j]
    
    mh_result = mantel_haenszel_dif(responses[:, j], group, score_without_item)
    lr_result = logistic_regression_dif(responses[:, j], group, score_without_item)
    
    results.append({
        'item': j + 1,
        'true_dif': j in true_dif_items,
        'mh_chi2': mh_result['chi2_mh'],
        'mh_p': mh_result['p_value'],
        'mh_d_dif': mh_result['mh_d_dif'],
        'lr_uniform_p': lr_result['uniform_dif']['p_value'],
        'lr_nonuniform_p': lr_result['nonuniform_dif']['p_value'],
        'lr_total_p': lr_result['total_dif']['p_value']
    })

results_df = pd.DataFrame(results)
results_df

In [ ]:
# DIFが検出された項目
alpha = 0.05

print("=== Mantel-Haenszel法で検出されたDIF項目 ===")
detected_mh = results_df[results_df['mh_p'] < alpha]['item'].tolist()
print(f"検出された項目: {detected_mh}")
print(f"真のDIF項目: {[i+1 for i in true_dif_items]}")

print("\n=== ロジスティック回帰法で検出されたDIF項目 ===")
detected_lr = results_df[results_df['lr_total_p'] < alpha]['item'].tolist()
print(f"検出された項目: {detected_lr}")
print(f"真のDIF項目: {[i+1 for i in true_dif_items]}")

## DIFの解釈と対処

DIFが検出された場合、以下のステップで対処する：

1. **統計的有意性の確認**: 多重比較の補正（Bonferroni補正など）を考慮
2. **効果量の評価**: MH D-DIFの絶対値が1.0以上で「中程度」、1.5以上で「大きい」とされる（ETS基準）
3. **項目内容の精査**: 専門家による項目レビューを実施し、DIFの原因を特定
4. **対処の決定**: 
   - 項目の削除
   - 項目の修正
   - 集団別の項目パラメータを使用
   - DIFの影響を考慮したスコアリング

## 参考文献

:::{card}

Holland, P. W., & Wainer, H. (Eds.). (1993). *Differential item functioning*. Lawrence Erlbaum Associates.

DIFに関する包括的な教科書

:::

:::{card}

Magis, D., Béland, S., Tuerlinckx, F., & De Boeck, P. (2010). [A general framework and an R package for the detection of dichotomous differential item functioning](https://doi.org/10.3758/BRM.42.3.847). Behavior Research Methods, 42(3), 847-862.

Rの`difR`パッケージの論文。DIF検出手法の比較も行っている

:::

:::{card}

Swaminathan, H., & Rogers, H. J. (1990). [Detecting differential item functioning using logistic regression procedures](https://doi.org/10.3102/10769986015002117). Journal of Educational Measurement, 27(4), 361-370.

ロジスティック回帰法によるDIF検出の基礎論文

:::